In [7]:
from Bio import Entrez
import time
import requests

In [9]:
Entrez.email = "andja13damnjanovic@gmail.com"
Entrez.api_key = "e24a5d5f305cab8c57add5f35abe6dd6ff09"  # Replace with your real API key

batch_size = 500
input_file = "../processedDatasets/SARS-CoV2/accessions.txt"
output_file = "sequences.fasta"

def read_accessions(file_path):
    with open(file_path) as f:
        return [line.strip() for line in f if line.strip()]
    

def fetch_batch(acc_batch):
    attempt = 0
    while attempt < 3:
        try:
            accession_string = ",".join(acc_batch)
            url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
            params = {
                "db": "protein",
                "id": accession_string,
                "rettype": "fasta",
                "retmode": "text",
                "api_key": Entrez.api_key
            }
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.text
            else:
                raise Exception(f"HTTP {response.status_code}: {response.text}")
        except Exception as e:
            attempt += 1
            print(f"Retrying batch due to error: {e}")
            time.sleep(5)
    return ""

def write_fasta(data, out_path):
    with open(out_path, "a") as out:
        out.write(data)

def main():
    accessions = read_accessions(input_file)
    total = len(accessions)
    print(f"Total accessions: {total}")

    for i in range(0, total, batch_size):
        batch = accessions[i:i + batch_size]
        print(f"Fetching batch {i}–{i + len(batch)}...")
        fasta_data = fetch_batch(batch)
        if fasta_data:
            write_fasta(fasta_data, output_file)
        time.sleep(0.2)  # Use 0.1 for 10/sec with API key

if __name__ == "__main__":
    main()

Total accessions: 4112783
Fetching batch 0–500...
Retrying batch due to error: HTTP 414: 
Retrying batch due to error: HTTP 414: 


KeyboardInterrupt: 